In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pickle
import datetime


In [ ]:
# !#wget https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz
# !ls
# !gunzip
# !bash

In [ ]:
tokenizer = Tokenizer()
data = open('/content/drive/Shareddrives/Sandalu Pabasara/data.txt').read()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print(tokenizer.word_index)
print(total_words)

{'the': 1, 'to': 2, 'and': 3, 'of': 4, 'he': 5, 'his': 6, 'in': 7, 'a': 8, 'that': 9, 'was': 10, 'it': 11, 'had': 12, 'as': 13, 'with': 14, 'for': 15, 'not': 16, 'on': 17, 'but': 18, 'at': 19, 'gregor': 20, 'she': 21, 'would': 22, 'her': 23, 'him': 24, 'they': 25, 'this': 26, 'from': 27, 'all': 28, 'is': 29, 'be': 30, 'or': 31, 'have': 32, 'room': 33, 'if': 34, 'could': 35, 'been': 36, 'there': 37, 'out': 38, 'by': 39, 'history': 40, 'you': 41, 'so': 42, "gregor's": 43, 'now': 44, 'father': 45, 'sister': 46, 'even': 47, 'one': 48, 'then': 49, 'into': 50, 'door': 51, 'their': 52, 'what': 53, 'no': 54, 'more': 55, 'are': 56, 'himself': 57, 'big': 58, 'mother': 59, 'were': 60, 'up': 61, 'about': 62, 'when': 63, 'any': 64, 'did': 65, 'back': 66, 'way': 67, 'project': 68, 'gutenberg': 69, 'work': 70, 'time': 71, 'them': 72, 'than': 73, 'other': 74, 'only': 75, 'an': 76, 'we': 77, 'do': 78, 'just': 79, 'some': 80, 'which': 81, 'much': 82, 'said': 83, 'i': 84, 'these': 85, 'first': 86, 'witho

In [ ]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(250))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
--0.7025

model = Sequential()
model.add(Embedding(total_words, 150, input_length=max_sequence_len-1))
model.add(LSTM(300))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
--0.7211

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(50))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
--Failed 0.05 long time

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 150, input_length=max_sequence_len-1))
model.add(LSTM(300))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
# model.add(Bidirectional(LSTM(150)))
# model.add(LSTM(50,return_sequences=True))
# model.add(Dense(50,activation='relu'))
# model.add(Dense(total_words, activation='relu'))
##Faild due to low accuracy

adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=150, verbose=1)
print(model.summary)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/150
905/905 [==============================] - 97s 105ms/step - loss: 6.4369 - accuracy: 0.0874
Epoch 2/150
905/905 [==============================] - 95s 105ms/step - loss: 5.4868 - accuracy: 0.1350
Epoch 3/150
905/905 [==============================] - 95s 105ms/step - loss: 4.8168 - accuracy: 0.1594
Epoch 4/150
905/905 [==============================] - 95s 105ms/step - loss: 4.2321 - accuracy: 0.1940
Epoch 5/150
905/905 [==============================] - 95s 105ms/step - loss: 3.7019 - accuracy: 0.2460
Epoch 6/150
905/905 [==============================] - 95s 105ms/step - loss: 3.2853 - accuracy: 0.2915
Epoch 7/150
905/905 [==============================] - 95s 104ms/step - loss: 3.0061 - accuracy: 0.3265
Epoch 8/150
905/905 [==============================] - 95s 105ms/step - loss: 2.7776 - accuracy: 0.3636
Epoch 9/150
905/905 [==============================] - 94s 104ms/step - loss: 2.6100 - accuracy: 0.3909
Epoch 10/150
905/905 [==============================] - 95s 105m

In [ ]:
filename = "model"+str(datetime.datetime.today().strftime("%d%m%y"))
model.save("saved_model/"+filename+"_0.7211")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
modelname = '/content/drive/Shareddrives/Sandalu Pabasara/model171221_0.7211'
model = tf.keras.models.load_model(modelname)
print(modelname+" loded.")
while True:
    seed_text = input("::")
    next_words = 10
    #Prediction
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=16, padding='pre')

    predicted = model.predict(token_list, verbose=0)
    classes=np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == classes:
          output_word = word
          break
    seed_text += " " + output_word
    print(seed_text)